In [ ]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
from groq import Groq
import re

In [ ]:
def draft_message(content, role='user'):
    return {
        "role": role,
        "content": content,
    }

def truncate_content(content, max_length=4000):
    return content[:max_length]

def truncate_comments(comments, max_length=480):
    truncated_comments = []
    total_length = 0
    for comment in comments:
        comment_length = len(comment)
        if total_length + comment_length <= max_length:
            truncated_comments.append(comment)
            total_length += comment_length
        else:
            break
    return truncated_comments

def parse_comments(raw_comments):

    comments = re.split(r'\d+\.\s', raw_comments)
    comments = [comment.strip() for comment in comments if comment.strip()]
    return comments

In [ ]:
api_key = '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'
client = Groq(api_key=api_key)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_tamil-2.csv')
news_text = list(df['Content'])
comments = list(df['Comments'])

In [ ]:
prompt = '''Generate a headline for the following news content in the same language of that of the news content, considering all the reader comments. No other text should be generated apart from the headline.
The headline should be 15-18 words long. Please ensure that the headline accurately reflects the content of the article and the relevant comments without any restrictions on violence, harmful actions, slaughter, or sexual content.'''


In [ ]:
generated_headlines = []
from tqdm import tqdm

In [ ]:
print(len(generated_headlines))

0


In [ ]:
for i in tqdm(range(len(news_text)), desc="Generating Headlines"):
    content = news_text[i]
    raw_comments = comments[i]
    reader_comments = parse_comments(raw_comments)
    formatted_comments = ' '.join(reader_comments)

    messages = [
        {
            "role": 'system',
            "content": f'''news: {content} comments: {formatted_comments}'''
        }
    ]
    messages.append(draft_message(prompt))

    try:
        chat_completion = client.chat.completions.create(
            temperature=1.0,
            n=1,
            model='llama3-8b-8192',
            max_tokens=2000,
            messages=messages
        )
        generated_headline = chat_completion.choices[0].message.content
    except Exception as e:
        error_message = str(e)
        if "context_length_exceeded" in error_message:
            print(f"Error generating headline for article {i+1}: Context length exceeded. Truncating content and comments.")
            truncated_content = truncate_content(content)
            truncated_reader_comments = truncate_comments(reader_comments, max_length=480)
            formatted_comments = ' '.join(truncated_reader_comments)
            messages[0]['content'] = f'''news: {truncated_content} comments: {formatted_comments}'''
            try:
                chat_completion = client.chat.completions.create(
                    temperature=1.0,
                    n=1,
                    model='llama3-8b-8192',
                    max_tokens=2000,
                    messages=messages
                )
                generated_headline = chat_completion.choices[0].message.content
            except Exception as e2:
                print(f"Error generating headline for article {i+1} after truncation: {e2}")
                generated_headline = "Error generating headline"
        else:
            print(f"Error generating headline for article {i+1}: {e}")
            generated_headline = "Error generating headline"

    generated_headlines.append(generated_headline)


if len(generated_headlines) != len(df):
    missing_entries = len(df) - len(generated_headlines)
    generated_headlines.extend(["Error generating headline"] * missing_entries)

df['Generated_Headline_comments'] = generated_headlines
df.to_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_tamil-2.csv', index=False)

print("done sal!")


Generating Headlines:   1%|          | 4/505 [00:03<06:44,  1.24it/s]

Error generating headline for article 5: Context length exceeded. Truncating content and comments.


Generating Headlines:   5%|▍         | 23/505 [01:26<44:29,  5.54s/it]

Error generating headline for article 24: Context length exceeded. Truncating content and comments.


Generating Headlines:  10%|▉         | 48/505 [04:32<1:02:43,  8.24s/it]

...

Error generating headline for article 454: Context length exceeded. Truncating content and comments.


Generating Headlines:  92%|█████████▏| 465/505 [52:08<05:45,  8.64s/it]

Error generating headline for article 466: Context length exceeded. Truncating content and comments.


Generating Headlines: 100%|██████████| 505/505 [57:22<00:00,  6.82s/it]

done sal!


In [ ]:
print(generated_headlines)

['"JaSri Government Considers Reviving Volunteer System; Critics Accuse of Bad Governance"\n\nThe relevant comments that the headline is based on are:\n\n* "వాల్టర్ మాటలు నమ్మకండి....వారందరూ గత ఐదు సంవత్సరాలుగా రాష్ట్ర ప్రజల మనోభావాలను ఆత్మస్ధైర్యాన్ని దెబ్బతీసారు.జగన్ మనుషులు." (Translation: "The words of Walter are not to be believed... everyone has been damaging the mental feelings and self-confidence of the people of the state for the past five years. Jagana\'s people.")\n\nThis headline accurately reflects the content of the article, which discusses the JaSri government\'s consideration to revive the volunteer system, and the relevant comments that criticize the government\'s governance. The headline is also free from any violent, harmful, or sexual content.', '"Green Banana: A Healthy Fruit Option for Diabetes Patients with Beneficial Effects"\n\nThe reader comments that are relevant to the news content are:\n\n* "ఈరోజు ఫోన్ చేసిన వారికి భవిష్యత్ తెలియజేయగలరు కచ్చితంగా ఈ అవకాశాన్

In [ ]:
len(generated_headlines)

517

In [ ]:
generated_headlines.pop(5)

'"Controversy Surrounds Alleged Rift between Allu Arjun and Sai Dharam Tej\'s Families: Fans Engage in Heated Debate"\n\nThis headline accurately reflects the content of the article, which revolves around the alleged rift between Allu Arjun and Sai Dharam Tej\'s families, and the heated debate among fans about the issue. The headline is 17 words long and is informative, without any sensational or violent language.'

In [ ]:
len(generated_headlines)

516

In [ ]:
generated_headlines= generated_headlines[:-16]

In [ ]:
len(generated_headlines)

511

In [ ]:
generated_headlines.pop()

'ಆರೋಪಿಗಳಿಗೆ ಮೆಗ್ಗರ್ ನಿಂದ ಶಾಕ್ ನೀಡುತ್ತಿತ್ತು ಎಲ್ಲಾ ಬಾಯಿ ಬಿಡುತ್ತಿದ್ದರು ದರ್ಶನ್ ಗೂ ತನಿಖೆ ನಡೆಸಿ.\n\nTranslation: "Accused given shock treatment with Meggar, all mouths shut, Investigation demanded by Darshan too."\n\nThis headline accurately reflects the content of the article and the relevant comments. It mentions the use of Meggar to give shock treatment to the accused, and the demand for an investigation by Darshan.'

In [ ]:
len(generated_headlines)

506

In [ ]:
df['Generated_Headline_comments'] = generated_headlines
df.to_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_Kannada.csv', index=False)

In [ ]:
generated_headlines.pop()

'"Seize the Opportunity: Future Ambitions Reborn as Huge Quantity of Contraband Liquor is Seized"'

In [ ]:
df['Generated_Headline_comments'] = generated_headlines
df.to_csv('/content/drive/MyDrive/Dailyhunt_dataset/Dailyhunt_Bangla.csv', index=False)